In [1]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Sequential
tf.compat.v1.enable_eager_execution()
tf.compat.v1.disable_v2_behavior()
import tensorflow.experimental.numpy as tnp
tf.enable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
lyrics = open("Lyrics_Counterparts.txt", encoding="utf-8").read().lower()

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(lyrics)

In [4]:
tokenizer.texts_to_sequences(["First"])

[[17, 5, 8, 9, 3]]

In [5]:
max_id = len(tokenizer.word_index) #number of distinct characters in text
dataset_size = tokenizer.document_count #total number of characters
print(max_id, " ", dataset_size)

52   71707


In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([lyrics])) - 1
#encodes the full text so each character is represented by its ID

In [7]:
#splitting dataset for training/test/validation
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
#convert long string of characters into windows for training(batches)
n_steps = 100
window_length = n_steps + 1 #target character = input shifted 1 char ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
#flatten the dataset for training
dataset = dataset.flat_map(lambda window: window.batch(window_length))
#this gives us a single tensor for each window

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [12]:
dataset = dataset.prefetch(1)

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                    dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)

Epoch 1/20
2014/2014 [==============================] - 683s 336ms/step - loss: 2.2633
Epoch 2/20
2014/2014 [==============================] - 676s 335ms/step - loss: 1.5648
Epoch 3/20
2014/2014 [==============================] - 676s 335ms/step - loss: 1.4516
Epoch 4/20
2014/2014 [==============================] - 674s 334ms/step - loss: 1.3973
Epoch 5/20
2014/2014 [==============================] - 676s 335ms/step - loss: 1.3619
Epoch 6/20
2014/2014 [==============================] - 673s 333ms/step - loss: 1.3355
Epoch 7/20
2014/2014 [==============================] - 679s 336ms/step - loss: 1.3174
Epoch 8/20
2014/2014 [==============================] - 676s 335ms/step - loss: 1.3007
Epoch 9/20
2014/2014 [==============================] - 669s 331ms/step - loss: 1.2858
Epoch 10/20
2014/2014 [==============================] - 676s 335ms/step - loss: 1.2748
Epoch 11/20
2014/2014 [==============================] - 675s 334ms/step - loss: 1.2654
Epoch 12/20
2014/2014 [==================

In [14]:
model.save('char_rnn_lyrics')

In [15]:
tf.executing_eagerly()

True

In [16]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [17]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

C:\Users\mitch\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'u'

In [18]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new,steps=1)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)/ temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
#this function picks the next character randomly, with a probability equal to the estimated probability,using tf.random.categorical
#categorical() samples random class indices, given the class probs (logits)
#temperature is a variables that controls the diversity of the characters generated
# a near-zero temperature will output characters with high probabilities
# a high temperature will give all characters equal probability

In [19]:
#this function repeatedly calls the above function to generate text
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char (text, temperature)
    return text

In [23]:
print(complete_text("a", temperature=1))

ains

this is the heart between nothing more can timely where i just from rorring to find you

any "r


In [24]:
print(complete_text("b", temperature=1))

beging
and we run from right among earth i am a god
caused me for right what i'll allow you i vill la


In [25]:
print(complete_text("f", temperature=1))

f than but i need souls of the fell that a dollection on our casualtyen
all i’vensle with from the pr
